In [69]:
import oracledb, os, time

In [70]:
# running oracle-free image

def create_connection():
    try:
        connection = oracledb.connect(
            user="SYSTEM",
            password=os.getenv("ORACLE_PASSWORD"),
            dsn="localhost:1523/FREE"          
        )
        print("Verbindung zur PDB erfolgreich!")
    except oracledb.DatabaseError as e:
        print(f"Verbindung fehlgeschlagen: {e}")
    return connection

# Queries

In [71]:
# Get all timestamps to determine data coverage and time range
time_stamps_query = """
SELECT TS FROM IOT_TELEMETRY_DATA
"""

In [72]:
# Calculate average humidity per device across all time periods
average_humidity_per_device_query = """SELECT
  DEVICE,
  AVG(HUMIDITY) AS avg_humidity
FROM IOT_TELEMETRY_DATA
GROUP BY DEVICE
ORDER BY DEVICE
"""

In [73]:
# Find the highest temperature recorded across all devices and time periods
highest_temp_query = """SELECT
  MAX(temp) AS max_temperature
FROM IOT_TELEMETRY_DATA
"""

In [74]:
# Calculate total air quality score (CO + LPG + Smoke) per device and rank by worst air quality
worst_air_quality_per_device_query = """SELECT
  device,
  SUM(co + lpg + smoke) AS air_quality_score
FROM IOT_TELEMETRY_DATA
GROUP BY device
ORDER BY air_quality_score DESC
"""

In [75]:
# Calculate average temperature to humidity ratio within a specific time window
avg_temp_humidity_ratio_query = """
SELECT
    AVG(TEMP / NULLIF(HUMIDITY, 0)) AS avg_temp_humidity_ratio
FROM
    IOT_TELEMETRY_DATA
WHERE
    TS >= TO_TIMESTAMP('2020-07-14 00:01:00', 'YYYY-MM-DD HH24:MI:SS')
    AND TS <= TO_TIMESTAMP('2020-07-19 03:01:00', 'YYYY-MM-DD HH24:MI:SS')
"""

In [76]:
# Calculate minute-by-minute temperature to humidity ratio per device for time-series analysis
avg_temp_humidity_ratio_per_min_oracle_query = """SELECT
         device,
         TRUNC(ts, 'MI') AS minute_bucket,
         AVG(temp / NULLIF(humidity, 0)) AS avg_temp_humidity_ratio_per_min
    FROM iot_telemetry_data
   WHERE ts >= TO_TIMESTAMP('2020-07-14 00:01:00', 'YYYY-MM-DD HH24:MI:SS')
     AND ts <  TO_TIMESTAMP('2020-07-19 03:01:00', 'YYYY-MM-DD HH24:MI:SS')
     AND humidity <> 0
GROUP BY device, TRUNC(ts, 'MI')
ORDER BY device, minute_bucket
"""

In [77]:
import pandas as pd
try:
    connection = create_connection()
    cursor = connection.cursor()
    cursor.execute(time_stamps_query)
    time_stamps = pd.DataFrame(cursor.fetchall())
    print(time_stamps.min())
    print(time_stamps.max())
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error reading data:", e)

Verbindung zur PDB erfolgreich!
0   2020-07-12 00:01:34
dtype: datetime64[ns]
0   2020-07-20 00:03:37
dtype: datetime64[ns]
0   2020-07-12 00:01:34
dtype: datetime64[ns]
0   2020-07-20 00:03:37
dtype: datetime64[ns]


# Performance Analysis

In [78]:
query_times = {}

In [79]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(average_humidity_per_device_query)
    avg_humidity = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(avg_humidity)
    query_times['Average Humidity'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error reading data:", e)

Verbindung zur PDB erfolgreich!
Successfull fetched in 0.10586690902709961 seconds.
[('00:0f:00:70:91:0a', 75.44436079832734), ('1c:bf:ce:15:ec:4d', 61.91024663855242), ('b8:27:eb:bf:9d:51', 50.814076745389464)]
Successfull fetched in 0.10586690902709961 seconds.
[('00:0f:00:70:91:0a', 75.44436079832734), ('1c:bf:ce:15:ec:4d', 61.91024663855242), ('b8:27:eb:bf:9d:51', 50.814076745389464)]


In [80]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(highest_temp_query)
    highest_temp = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(highest_temp)
    query_times['Highest Temp'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)


Verbindung zur PDB erfolgreich!


Successfull fetched in 0.05444169044494629 seconds.
[(30.600000381469727,)]


In [81]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(worst_air_quality_per_device_query)
    worst_air = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(worst_air)
    query_times['Worst Air'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)

Verbindung zur PDB erfolgreich!


Successfull fetched in 0.11015534400939941 seconds.
[('b8:27:eb:bf:9d:51', 6777.162111500038), ('1c:bf:ce:15:ec:4d', 3054.864279412305), ('00:0f:00:70:91:0a', 2785.2340317640083)]


In [82]:

try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(avg_temp_humidity_ratio_query)
    worst_air = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(worst_air)
    query_times['Avg Temp Humidity Ratio'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)

Verbindung zur PDB erfolgreich!


Successfull fetched in 0.156602144241333 seconds.
[(0.39024171766859717,)]


In [83]:
try:
    connection = create_connection()
    cursor = connection.cursor()
    start_time = time.time()
    cursor.execute(avg_temp_humidity_ratio_per_min_oracle_query)
    worst_air = cursor.fetchall()
    end_time = time.time()
    print(f"Successfull fetched in {end_time - start_time} seconds.")
    print(worst_air)
    query_times['Avg Temp Humidity Ratio per Minute'] = (end_time - start_time)
    cursor.close()
    connection.close()
except oracledb.DatabaseError as e:
    print("Error: ", e)

Verbindung zur PDB erfolgreich!
Successfull fetched in 0.5675630569458008 seconds.
[('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 1), 0.2544215488395143), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 2), 0.25511925249735395), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 3), 0.25497148337630626), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 4), 0.25026099953875447), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 5), 0.25490973885315676), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 6), 0.25551907291367737), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 7), 0.25620766315467913), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 8), 0.25614734484547924), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 9), 0.2563328862822797), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 10), 0.25706594343158445), ('00:0f:00:70:91:0a', datetime.datetime(2020, 7, 14, 0, 11), 0.2578148766760841), ('00:0f:00:70:9

# Summary

In [84]:
performance = "OracleDB query times: " + str(query_times)
print(performance)

OracleDB query times: {'Average Humidity': 0.10586690902709961, 'Highest Temp': 0.05444169044494629, 'Worst Air': 0.11015534400939941, 'Avg Temp Humidity Ratio': 0.156602144241333, 'Avg Temp Humidity Ratio per Minute': 0.5675630569458008}


In [85]:
filename = "performance.txt"

with open(filename, "a", encoding="utf-8") as f:
    f.write(performance + "\n")

print(performance)


OracleDB query times: {'Average Humidity': 0.10586690902709961, 'Highest Temp': 0.05444169044494629, 'Worst Air': 0.11015534400939941, 'Avg Temp Humidity Ratio': 0.156602144241333, 'Avg Temp Humidity Ratio per Minute': 0.5675630569458008}
